# 其他增强（llama-index 版 rag fusion，混合检索，重写查询）

本章是对部分前置阶段的不同框架实现，看过前几个阶段的读者可以跳过

## 0. llama-index 简介

Llama-Index的核心组件主要由一下这几个组件组成：
- 向量存储索引（VectorStoreIndex）**：用于存储文档的向量表示，以便快速检索。
- 摘要索引（SummaryIndex）**：存储文档的摘要信息，提供快速预览和检索。
- 对象索引（ObjectIndex）**：索引文档中的具体对象或实体，以便进行详细的信息检索。
- 查询引擎工具（QueryEngineTool）**：提供强大的查询处理能力，支持复杂的检索任务。
- OpenAI代理（OpenAIAgent）**：集成OpenAI技术，增强生成能力和智能交互。
- FnRetrieverOpenAI代理（FnRetrieverOpenAIAgent）**：结合检索与生成的功能，提供更丰富的用户体验。

可以简单跑下如下示例，感受一下llamaindex，

> 以下基于 llamaindex 版本 V0.10.26

In [3]:
import os
os.environ['OPENAI_API_KEY'] = "sk-你的sk"

In [18]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('./data/mutli_documents_data').load_data()
index = VectorStoreIndex.from_documents(documents)

In [19]:
query_engine = index.as_query_engine()
response = query_engine.query("王奇")
print(response)

There is no information or mention of "王奇" in the provided context.


In [20]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# 检查存储是否已存在
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # 加载文档并创建索引
    documents = SimpleDirectoryReader("./data/mutli_documents_data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # 存储起来以备后用
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # 加载现有索引
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# 查询索引
query_engine = index.as_query_engine()
response = query_engine.query("说一下王奇")
print(response)

王奇是本书的作者之一，他在书中提到了乔布斯的例子，强调了学习强化学习的重要性，并分享了乔布斯在大学学习书法课的经历。


## 一、Rag fusion


### 1、背景

​       传统Rag技术能够降低大模型的幻觉并且结合最新的文档知识来进行回答,这种技术的出现，为人工智能和搜索领域带来了一场革命，它通过结合向量检索与生成算法的能力，实现了一种全新的交互方式。尽管如此，这项创新技术也存在一些挑战。因此需要引入高级Rag技术

​       在RAG的框架下，搜索环节不可或缺，这一环节本身却面临着诸多难题：提升向量召回的精确度可以结合传统的BM25算法和向量检索技术来共同完成搜索任务。然而，即便如此，有时期望的结果仍可能未被检索到。

​        在这样的背景下，我们可以从传统搜广推技术中汲取灵感，寻求可能的改进空间。在实际的检索操作之前，有一个关键步骤是查询词处理与文本理解，这一步也可以运用众多自然语言处理（NLP）技术，如分词、命名实体识别（NER）、同义词处理、词项权重分配、意图识别以及查询词的改写等一系列复杂的NLP任务，现在通过大模型，我们可以相对方便的运用上述方式来增强检索信息的丰富性，从而提高搜索的准确性。

​        那么怎么利用大语言模型来优化查询词处理与文本理解这一关键步骤呢，接下来，我会先介绍的基本的RAG-Fusion技术，这种技术正是基于llm对查询query进行改写，来增强检索信息。并且后面还会着重介绍llamaindex实现的几种改进的RAG-Fusion技术。

### 2.Rag fusion

​       RAG-fusion 简而言之就是生成多个用户查询并对结果重新排序利用倒数排名融合(RRF)和自定义向量得分加权来获得全面、准确的结果。首先利用 LLM 把用户原始查询query翻译成成N个不同但是语义相似的查询query，然后分别进行检索，接着将这多路召回的结果重排生成结果。
如下图所示：

![rrfsample](./figures/rrfsample.png)



下面再简单介绍一下倒数排序融合：

倒数排序融合（Reciprocal Rank Fusion, RRF）是一种用于融合多个搜索系统或检索系统结果的方法，旨在通过综合多个排名列表来提高信息检索的准确性和效果。这种方法基于一个简单的原理，即假设所有系统的输出都是有价值的，而且对每个系统的贡献给予平等的重视。RRF特别适合于那些需要从不同数据源或搜索引擎中获取信息并将其整合为单一结果列表的场景，例如学术文献搜索、元搜索引擎等。

### RRF的工作原理

倒数排序融合通过为每个检索结果分配一个基于其在各个系统中排名的倒数得分来工作。然后，对同一结果的所有得分进行汇总，得到一个综合得分，最后根据这些综合得分对所有结果重新排序，以产生最终的融合结果列表。

RRF的基本公式为：
$$
RRF(S) = \sum_{i=1}^{N} \frac{1}{k + rank_i(S)}
$$

- 其中，*S*代表某个具体的搜索结果。
- rank*i*(*S*)是搜索结果S*在第i*个系统中的排名。
- *N*是系统的总数。
- *k*是一个常数，用于调整排名的权重（通常设置为60）。

### 示例

假设我们有两个搜索系统（A和B），并对同一个查询请求产生了结果。我们想要使用RRF方法来融合这两个系统的结果。以下是每个系统返回的前三个结果及其排名：

- **系统A的结果**:
  - 文档1: 排名1
  - 文档2: 排名2
  - 文档3: 排名3
- **系统B的结果**:
  - 文档2: 排名1
  - 文档1: 排名2
  - 文档4: 排名3

我们假设*k*=60，根据RRF公式计算每个文档的融合得分。

- **文档1的得分**: 1/(60+1)+1/(60+2)==0.03265
- **文档2的得分**: 1/(60+2)+1/(60+1)==0.03265
- **文档3的得分**: 1/(60+3)==0.01613
- **文档4的得分**: 1/(60+3)==0.01613

最后，我们根据这些融合得分对文档进行重新排序。在这个例子中，文档1和文档2有相同的最高得分，因此它们将共享最高排名位置。文档3和文档4的得分较低，它们将排在后面。

01. 下面会用llama-index进行实操，首先构建一个字典，包含文件的八个标准问答对，key是问题，value是期望rag的答案

In [32]:
qna_dict = {
        "介绍下SVM算法": "是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机",
        "用通俗的语言介绍下强化学习": "监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略",
        "BN 和 LN 区别": "Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。",
        "讲讲self attention": "Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系",
        "Bert 的预训练过程": "Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。",
        "GPT 与 Bert 的区别": "GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。",
        "pca 属于有监督还是无监督": "PCA 按有监督和无监督划分应该属于无监督学习，所以数据集有无 y 并不重要，只是改变样本 X 的属性(特征)维度。",
        "介绍 transformer 算法": "Transformer 本身是一个典型的 encoder-decoder 模型，Encoder 端和 Decoder 端均有 6 个 Block，Encoder 端的 Block 包括两个模块，多头 self-attention 模块以及一个前馈神经网络模块；Decoder 端的Block 包括三个模块，多头 self-attention 模块，多头 Encoder-Decoder attention 交互模块，以及一个前馈神经网络模块；需要注意：Encoder 端和 Decoder 端中的每个模块都有残差层和 Layer Normalization层。",
    }

02. 然后构建评估prompt和评估函数：


意在让大模型判断生成的结果和期望的结果是否含义一致，在下面使用各种advanecd-rag方法时会经常调用， 主要使用Llama Packs，Llama Packs是对llamaindex的更加高级的封装，几行代码实现各种高级RAG，如果想看具体的流程也可以直接进入github查看源码

In [36]:
from llama_index.core.prompts import PromptTemplate
import re
import nest_asyncio
nest_asyncio.apply()
eval_template = PromptTemplate("""
            输入:
            Question: {question}
            LLM-Answer: {llm_answer}
            Expected-Answer: {expected_answer}

            任务:
            比较LLM-Answer和Expected-Answer，确定它们是否传达类似的含义。

            输出:
            意思一致：如果LLM-Answer和Expected-Answer传达类似的含义。 
            意思不同：如果LLM-Answer和Expected-Answer传达完全不同的含义。

            示例： 
            Question：法国的首都是什么？ 
            LLM-Answer: 巴黎 
            Expected-Answer: 光之城

            输出:
            意思一致""")
    
def eval_llm_output(row, llm):
    ''' Eval llm_answer and expected_answer for the question with llm '''
    question = row['question']
    llm_answer = row['llm_answer']
    expected_answer = row['expected_answer']
    prompt_message = eval_template.format(question=question, llm_answer=llm_answer, expected_answer=expected_answer)
    answer = llm.complete(prompt_message).text
    return answer.strip()

初始化 llm

In [3]:

from llama_index.llms.openai import OpenAI

llm = OpenAI()

载入需要查询的数据文件

In [4]:
from llama_index.readers.file import PDFReader
from pathlib import Path

loader = PDFReader()
documents = loader.load_data(
        file=Path(f"./data/face.pdf")
    )


In [37]:
#pdf的页数
len(documents)

46

In [40]:
from llama_index.core.node_parser import SimpleNodeParser
node_parser = SimpleNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(documents)

In [41]:
len(nodes)

46

## 二、混合融合检索HybridFusionRetriever:(原理类似原本的Rag-fusion)
1. 根据输入问题Query重写翻译为多个相似语义的Query
2. 接着进行向量Embeding(vector)+关键词搜索(bm25 search)
3. 接着用Reciprocal Rank Fusion(倒数排序融合)对搜索结果进行重排，选择topk结果输入llm
    
### bm25

这里简单说一下bm25：

BM25 是一种在信息检索领域广泛使用的排名函数，主要用于评估文档与用户查询之间的相关性。它是基于概率检索框架的一种改进，通过考虑查询中的词项在文档中的频率（Term Frequency, TF）以及在整个文档集合中的逆文档频率（Inverse Document Frequency, IDF）来计算。

#### 词项频率 (TF)
词项在文档中出现的次数。BM25 对 TF 进行了饱和处理，以避免过分强调在文档中频繁出现的词项。
#### 逆文档频率 (IDF)
衡量一个词项在文档集合中的普遍重要性。如果一个词项在很多文档中都出现，则其 IDF 值会降低，反映其区分文档的能力较弱。
#### 文档长度归一化
文档的长度会影响词项频率的计算。BM25 通过文档长度归一化来防止长文档被过高估计相关性。
#### BM25 得分公式
BM25 的得分公式如下所示：
$$ \text{Score}(D, Q) = \sum_{i=1}^{n} IDF(q_i) \cdot \frac{f(q_i, D) \cdot (k_1 + 1)}{f(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})} $$
其中：
- $ D $ 是文档，
- $ Q $ 是查询，
- $ q_i $ 是查询 $ Q $ 中的第 $ i $ 个词项，
- $ f(q_i, D) $ 是词项 $ q_i $ 在文档 $ D $ 中的频率，
- $ |D| $ 是文档 $ D $ 的长度，
- $ \text{avgdl} $ 是文档集合中文档平均长度，
- $ k_1 $ 和 $ b $ 是可调的参数。
#### BM25 的优点
- **简单高效**：BM25 不需要复杂的预处理，计算速度快。
- **广泛应用**：由于其高效性，BM25 在工业界和学术界都有广泛应用。


In [43]:
from llama_index.core.llama_pack import download_llama_pack

HybridFusionRetrieverPack = download_llama_pack(
    "HybridFusionRetrieverPack",
    "./hybrid_fusion_pack",
)

In [44]:
hybrid_fusion_pack = HybridFusionRetrieverPack(
    nodes, chunk_size=256, vector_similarity_top_k=2, bm25_similarity_top_k=2
)

In [47]:
import pandas as pd
import nest_asyncio
nest_asyncio.apply()
import os
from getpass import getpass
answers = {'question':[], 'llm_answer': [], 'expected_answer': []}

for question in qna_dict.keys():
    expected_answer = qna_dict[question]
    # result
    response = hybrid_fusion_pack.run(question)
    answers['question'].append(question)
    answers['llm_answer'].append(response.response)
    answers['expected_answer'].append(expected_answer)

answers_df = pd.DataFrame(answers)

Generated queries:
1. SVM算法原理及应用领域介绍
2. SVM算法与其他机器学习算法的比较分析
3. SVM算法在数据挖掘和模式识别中的应用案例展示
Generated queries:
1. What is reinforcement learning explained in simple terms?
2. Easy explanation of reinforcement learning
3. Introduction to reinforcement learning in layman's terms
Generated queries:
1. BN vs LN comparison
2. What are the differences between BN and LN?
3. Features of BN and LN compared
Generated queries:
1. What is self attention in neural networks?
2. How does self attention improve natural language processing models?
3. Examples of self attention mechanisms in deep learning architectures.
Generated queries:
1. Bert 预训练模型的优势和劣势
2. Bert 预训练过程中使用的数据集
3. Bert 预训练模型的应用领域和效果
Generated queries:
1. GPT vs Bert: 什么是GPT和Bert，它们之间有什么区别？
2. GPT和Bert的不同之处是什么？
3. GPT和Bert在自然语言处理中有何异同？
Generated queries:
1. What is the difference between supervised and unsupervised learning in machine learning?
2. Examples of supervised learning algorithms in machine learning.
3. How does principal component

我们可以简单看下检索的结果

In [48]:
import pandas as pd

with pd.option_context('display.max_colwidth', None):
    display(answers_df)


,question,llm_answer,expected_answer
0,介绍下SVM算法,"SVM is a binary classification model that defines the maximum margin linear classifier in the feature space. It distinguishes itself from the perceptron by maximizing the margin. SVM can be categorized into three types: linear separable SVM for linearly separable training data, linear SVM for data that is not linearly separable but approximately so, and non-linear SVM for data that is not linearly separable. The learning strategy of SVM involves maximizing the margin, which can be formalized as solving a convex quadratic programming problem or minimizing the regularized hinge loss function. The learning algorithm of SVM focuses on solving convex quadratic programming optimization. Additionally, the choice of kernel function in SVM includes Linear kernel for linearly separable cases and RBF kernel for non-linearly separable cases, with the latter requiring parameter tuning through methods like cross-validation.",是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机
1,用通俗的语言介绍下强化学习,强化学习是一种机器学习方法，通过代理程序在与环境的交互中学习如何做出一系列决策，以使得获得的累积奖励最大化。在这个过程中，代理程序会根据环境的反馈调整自己的行为，从而逐渐学会选择最优的行动策略。强化学习的目标是使代理程序能够在不断尝试和学习中获得最大的长期回报。,监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略
2,BN 和 LN 区别,BatchNormalization是对这批样本的同一维度特征做归一化，LayerNormalization是对这单个样本的所有维度特征做归一化。LN中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN中则针对不同神经元输入计算均值和方差，同一个batch中的输入拥有相同的均值和方差。因此，LN不依赖于batch的大小和输入sequence的长度，可以用于batchsize为1和RNN中sequence的normalize操作。,Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。
3,讲讲self attention,"SelfAttention is a mechanism that differs significantly from traditional Attention. While traditional Attention calculates dependencies between hidden states of the source and target ends, SelfAttention operates separately on the source and target ends. It focuses solely on the self-relevance of the source or target input, capturing dependencies between words within the source or target end. The self-attention results from the source end are then incorporated into the attention obtained from the target end, capturing dependencies between words in both the source and target ends. This unique approach allows SelfAttention to capture dependencies not only between words in the source and target ends but also within the source or target end itself, making it more effective than traditional Attention mechanisms.",Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系
4,Bert 的预训练过程,BERT的预训练过程使用了LAMB优化器，该优化器适用于小批量和大批量训练，并且不需要调整除学习率以外的其他超参数。通过LAMB优化器，BERT的预训练批量大小可以扩展到64K，而不会损失准确率，从而使BERT的训练过程可以在76分钟内完成。,Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。
5,GPT 与 Bert 的区别,GPT是单向模型，只能利用上文信息，而BERT是双向模型，可以同时利用上下文信息。,GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。
6,pca 属于有监督还是无监督,PCA belo

用 `eval_llm_output` 来评估 llm-answer 和 expected-answer 是否一致

In [ ]:
answers_df['rag_eval_results'] = answers_df.apply(lambda r: eval_llm_output(r, llm), axis=1)

In [70]:
with pd.option_context('display.max_colwidth', None):
    display(answers_df)

,question,llm_answer,expected_answer,rag_eval_results
0,介绍下SVM算法,"SVM is a binary classification model that defines the maximum margin linear classifier in the feature space. It distinguishes itself from the perceptron by maximizing the margin. SVM can be categorized into three types: linear separable SVM for linearly separable training data, linear SVM for data that is not linearly separable but approximately so, and non-linear SVM for data that is not linearly separable. The learning strategy of SVM involves maximizing the margin, which can be formalized as solving a convex quadratic programming problem or minimizing the regularized hinge loss function. The learning algorithm of SVM focuses on solving convex quadratic programming optimization. Additionally, the choice of kernel function in SVM includes Linear kernel for linearly separable cases and RBF kernel for non-linearly separable cases, with the latter requiring parameter tuning through methods like cross-validation.",是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机,意思一致。LLM-Answer和Expected-Answer都在介绍SVM算法是一种二分类模型，定义了特征空间上的最大间隔线性分类器。两者都提到了SVM通过最大化间隔来区分自己与感知机的不同之处。因此，它们传达的含义是类似的。
1,用通俗的语言介绍下强化学习,强化学习是一种机器学习方法，通过代理程序在与环境的交互中学习如何做出一系列决策，以使得获得的累积奖励最大化。在这个过程中，代理程序会根据环境的反馈调整自己的行为，从而逐渐学会选择最优的行动策略。强化学习的目标是使代理程序能够在不断尝试和学习中获得最大的长期回报。,监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略,意思不同
2,BN 和 LN 区别,BatchNormalization是对这批样本的同一维度特征做归一化，LayerNormalization是对这单个样本的所有维度特征做归一化。LN中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN中则针对不同神经元输入计算均值和方差，同一个batch中的输入拥有相同的均值和方差。因此，LN不依赖于batch的大小和输入sequence的长度，可以用于batchsize为1和RNN中sequence的normalize操作。,Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。,意思一致。LLM-Answer和Expected-Answer传达的含义是一致的，只是表达方式略有不同。都在解释BatchNormalization和LayerNormalization的区别，以及它们各自的特点。
3,讲讲self attention,"SelfAttention is a mechanism that differs significantly from traditional Attention. While traditional Attention calculates dependencies between hidden states of the source and target ends, SelfAttention operates separately on the source and target ends. It focuses solely on the self-relevance of the source or target input, capturing dependencies between words within the source or target end. The self-attention results from the source end are then incorporated into the attention obtained from the target end, capturing dependencies between words in both the source and target ends. This unique approach allows SelfAttention to capture dependencies not only between words in the source and target ends but also within the source or target end itself, making it more effective than traditional Attention mechanisms.",Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系,意思不同。LLM-Answer和Expected-Answer虽然都在讲Self Attention与传统Attention的区别，但表达方式和详细内容有所不同。LLM-Answer更加强调Self Attention的独特之处和优势，而Expected-Answer则更加详细地解释了Self Attention的工作原理和优势。
4,Bert 的预训练过程,BERT的预训练过程使用了LAMB优化器，该优化器适用于小批量和大批量训练，并且不需要调整除学习率以外的其他超参数。通过LAMB优化器，BERT的预训练批量大小可以扩展到64K，而不会损失准确率，从而使BERT的训练过程可以在76分钟内完成。,Bert 的预训练主要包含两个任务，MLM 和 NSP，Ma

将结果写入 excel 便于分析

In [73]:
answers_df.to_excel('hybrid_fusion_pack_results.xlsx', index=False)

## 三、重写查询检索Query Rewriting Retriever
1. 根据输入问题Query重写翻译为多个相似语义的Query，默认生成多个
2. 接着进行向量Embeding(vector)搜索 
3. 接着用Reciprocal Rank Fusion(倒数排序融合)对搜索结果进行重排，选择topk结果输入llm

首先下载并构建 query 改写 pack

In [75]:
# # 下载并初始化 Pack
from llama_index.core.llama_pack import download_llama_pack

QueryRewritingRetrieverPack = download_llama_pack(
    "QueryRewritingRetrieverPack",
    "./query_rewriting_pack",
    # leave the below commented out (was for testing purposes)
    # llama_hub_url="https://raw.githubusercontent.com/run-llama/llama-hub/jerry/add_llama_packs/llama_hub",
)

query_rewriting_pack = QueryRewritingRetrieverPack(
    nodes,
    chunk_size=256,
    vector_similarity_top_k=2,
)

用改写后的 query 进行检索

In [76]:
import pandas as pd

answers = {'question':[], 'llm_answer': [], 'expected_answer': []}

for question in qna_dict.keys():
    expected_answer = qna_dict[question]
    # result
    response = query_rewriting_pack.run(question)
    answers['question'].append(question)
    answers['llm_answer'].append(response.response)
    answers['expected_answer'].append(expected_answer)

answers_df = pd.DataFrame(answers)

Generated queries:
1. SVM算法原理及应用领域介绍
2. SVM算法与其他机器学习算法的比较分析
3. SVM算法在数据挖掘和模式识别中的实际应用案例
Generated queries:
1. What is reinforcement learning explained in simple terms?
2. Explain reinforcement learning in layman's terms
3. Introduction to reinforcement learning for beginners
Generated queries:
1. BN vs LN comparison
2. Key differences between BN and LN
3. What sets BN apart from LN?
Generated queries:
1. What is self attention in deep learning?
2. How does self attention improve natural language processing models?
3. Examples of self attention mechanisms in neural networks.
Generated queries:
1. Bert 预训练模型有哪些？
2. Bert 预训练算法是如何工作的？
3. Bert 预训练的优势和应用领域是什么？
Generated queries:
1. GPT vs Bert: 语言模型的不同之处
2. GPT 和 Bert 在自然语言处理中的应用比较
3. GPT 和 Bert 的性能对比研究结果
Generated queries:
1. 什么是主成分分析（PCA）？
2. PCA如何在有监督学习中使用？
3. PCA和无监督学习的区别是什么？
Generated queries:
1. Transformer算法原理介绍
2. Transformer算法应用领域
3. Transformer算法与其他深度学习算法比较


用 `eval_llm_output` 来评估 llm-answer 和 expected-answer 是否一致

In [78]:
# eval llm-answer with expected-answer
answers_df['rag_eval_results'] = answers_df.apply(lambda r: eval_llm_output(r, llm), axis=1)

with pd.option_context('display.max_colwidth', None):
    display(answers_df)
    
answers_df.to_excel('query_rewriting_pack_results.xlsx', index=False)

,question,llm_answer,expected_answer,rag_eval_results
0,介绍下SVM算法,SVM算法是一种二分类模型，其基本模型是定义在特征空间上的间隔最大的线性分类器。SVM可分为三种：线性可分SVM、线性SVM和非线性SVM。线性可分SVM适用于训练数据线性可分的情况，通过最大化硬间隔学习得到线性分类器；线性SVM适用于训练数据不能线性可分但近似线性可分的情况，通过最大化软间隔学习得到线性分类器；非线性SVM适用于训练数据线性不可分的情况，通过核技巧和最大化软间隔学习得到非线性分类器。,是一种二分类模型，它的基本模型是定义在特征空间上的间隔最大的线性分类器，间隔最大使它有别于感知机,意思不同
1,用通俗的语言介绍下强化学习,强化学习是一种机器学习方法，就像教小孩子学习一样。在强化学习中，模型通过尝试不同的行动来学习，然后根据行动的结果来做出调整。这种学习方式就像小孩子通过尝试不同的行为来学习什么是对的，什么是错的一样。强化学习的目标是让模型在特定环境中学会采取最优的行动，以获得最大的奖励。,监督学习的特点是有一个“老师”来“监督”我们，告诉我们正确的结果是什么。在我们在小的时候，会有老师来教我们，本质上监督学习是一种知识的传递，但不能发现新的知识。对于人类整体而言，真正（甚至唯一）的知识来源是实践——也就是强化学习。比如神农尝百草，最早人类并不知道哪些草能治病，但是通\过尝试，就能学到新的知识。学习与决策者被称为智能体，与智能体交互的部分则称为环境。智能体与环境不断进行交互，具体而言，这一交互的过程可以看做是多个时刻，每一时刻，智能体根据环境的状态，依据一定的策略选择一个动作（这里的策略指的是从环境状态到智能体动作或者动作概率之间的映射），然后环境依据一定的状态转移概率转移到下一个状态，与此同时根据此时状态的好坏反馈给智能体一个奖励。智能体可以根据环境的反馈调整其策略，然后继续在环境中探索，最终学习到一个能够获得最多奖励的最优策略,意思不同
2,BN 和 LN 区别,BatchNormalization是对不同神经元输入计算均值和方差，同一个batch中的输入拥有相同的均值和方差；LayerNormalization是对单个样本的所有维度特征做归一化，同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差。 LN不依赖于batch的大小和输入sequence的长度，因此可以用于batchsize为1和RNN中sequence的normalize操作。,Batch Normalization 是对这批样本的同一维度特征做归一化， Layer Normalization 是对这单个样本的所有维度特征做归一化。区别：LN 中同层神经元输入拥有相同的均值和方差，不同的输入样本有不同的均值和方差；BN 中则针对不同神经元输入计算均值和方差，同一个 batch 中的输入拥有相同的均值和方差。所以，LN 不依赖于batch 的大小和输入 sequence 的长度，因此可以用于 batchsize 为 1 和 RNN 中 sequence 的 normalize 操作。,意思不同
3,讲讲self attention,Transformer模型中的self-attention机制是一种注意力机制，用于计算输入序列中每个位置的表示与其他位置的关联程度。在self-attention中，每个输入位置都会与其他位置进行加权组合，以便更好地捕捉输入序列内部的依赖关系。这种机制允许模型在处理序列数据时聚焦于不同位置之间的交互，有助于提高模型对长距离依赖关系的建模能力。,Self Attention 与传统的 Attention 机制非常的不同：传统的 Attention 是基于 source 端和 target 端的隐变量（hidden state）计算 Attention 的，得到的结果是源端的每个词与目标端每个词之间的依赖关系。但Self Attention 不同，它分别在 source 端和 target 端进行，仅与 source input 或者 target input 自身相关的 Self Attention，捕捉 source 端或 target 端自身的词与词之间的依赖关系；然后再把 source 端的得到的self Attention 加入到 target 端得到的 Attention 中，捕捉 source 端和 target 端词与词之间的依赖关系。因此，self Attention Attention 比传统的 Attention mechanism 效果要好，主要原因之一是，传统的Attention 机制忽略了源端或目标端句子中词与词之间的依赖关系，相对比，self Attention 可以不仅可以得到源端与目标端词与词之间的依赖关系，同时还可以有效获取源端或目标端自身词与词之间的依赖关系,意思不同
4,Bert 的预训练过程,"During the pre-training process of Bert, two main objectives are utilized: Masked Language Model (MLM) and Next Sentence Prediction (NSP). The model is trained to predict masked words within a sentence using the MLM task, where random tokens are masked and the model is tasked with predicting them. Additionally, the NSP task involves determining whether a given pair of sentences appear consecutively in the original text or not. These two objectives are jointly trained to minimize a combined loss function, enhancing the model's ability to understand and generate language.",Bert 的预训练主要包含两个任务，MLM 和 NSP，Masked Language Model 任务可以理解为完形填空，随机 mask 每一个句子中 15%的词，用其上下文来做预测；Next Sentence Prediction 任务选择一些句子对 A 与 B，其中 50%的数据 B 是 A 的下一条句子，剩余 50%的数据 B 是语料库中随机选择的，学习其中的相关性。BERT 预训练阶段实际上是将上述两个任务结合起来，同时进行，然后将所有的 Loss 相加。,意思一致。LLM-Answer和Expected-Answer都在描述Bert的预训练过程，提到了MLM和NSP这两个主要任务，以及它们的具体实现方式和训练过程。两者传达的信息是相似的，只是表达方式略有不同。
5,GPT 与 Bert 的区别,GPT是单向模型，只能利用上文信息，而BERT是双向模型。 GPT基于自回归模型，适用于NLU和NLG任务，而BERT基于自编码模型，主要用于NLU任务，无法直接应用在文本生成上。,GPT 是单向模型，无法利用上下文信息，只能利用上文；而 BERT 是双向模型。GPT 是基于自回归模型，可以应用在 NLU 和 NLG 两大任务，而原生的 BERT 采用的基于自编码模型，只能完成 NLU 任务，无法直接应用在文本生成上面。,意思一致。LLM-Answer和Expected-Answer都提到了GPT是单向模型，无法利用上下文信息，只能利用上文；而BERT是双向模型。同时也提到了GPT基于自回归模型，适用于NLU和NLG任务，而BERT基于自编码模型，主要用于NLU任务，无法直接应用在文本生成上。因此它们传达的含义是相似的。
6,pca 属于有监督还是无监督,PCA belongs to unsupervised learning.,PCA 按有监督和无监督划分应该属于无监督学习，所以数据集有无 y 并不重要，只是改变样本 X 的属性(特征)维度。,意思一致。LLM-Answer和Expected-Answer都表明PCA属于无监督学习。虽然LLM-Answer没有提到数据集中y的重要性，但两者都强调PCA是用来改变样本X的属性维度的。因此，它们传达的含义是类似的。
7,介绍 transformer 算法,Transformer算法是一个典型的encoder-decoder模型，其中包括Encoder端和Decoder端，每个端都由多个Block组成。Encoder端包括多头self-attention模块和前馈神经网络模块，而Decoder端包括多头self-attention模块、多头Encoder-Decoder attention交互模块和前馈神经网络模块。在Transformer中，每个模块都包含残差连接和Layer Normalization层，这些组件共同构成了Transformer模型的核心结构。,Transformer 本身是一个典型的 encoder-decoder 模型，Encoder 端和 Decoder 端均有 6 个 Block，Encoder 端的 Block 包括两个模块，多头 self-attent

参考链接：https://docs.llamaindex.ai/en/v0.10.33/